# Machine Learning 2023-2024 - UMONS

# Model selection using scikit-learn

**During the last lab, we learned how to fit regression and classification models to a dataset with scikit-learn. However, by fixing the number of features and by fixing the model's hyperparameters beforehand, we restricted ourselves to a single model. By doing so, we omitted to explore a broader range of models, one of which might better explain the relationship between our input and target variables.**

**In this lab, we'll experiment with the general methodology of model selection, meaning that we'll define a set of predefined models, and we'll retain the one that minimizes the out-of-sample error.**

**Import the necessary libraries**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder

**In this lab, we will work with the [Fish market](https://www.kaggle.com/datasets/vipullrathod/fish-market) dataset, which contains several characteristics about fish, such as their weights, lengths, and species.**

In [2]:
df = pd.read_csv('data/fish_lab.csv', index_col=0)

**1) Print the type of each column and change the type of 'Species' to `category`.**

**We will start by predicting the target 'Height' from the feature 'Weight'. We split the dataset into a training and test set following a 75/25 partition using the `train_test_split` function. To avoid data leakage, we will not look at the test dataset during model selection.**

In [4]:
X, y = df[['Weight']], df['Height']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.75, test_size=0.25, shuffle=True, random_state=1
)

**2) What is the number of missing values? Using the `SimpleImputer` class of scikit-learn with the argument `strategy`, replace the missing values by the sample mean, computed only on the training dataset to avoid data leakage.**

**3) Generate a scatter plot of the two variables on the training set.**

**4) We can see that a linear model would not be the best option to model the relationship between these variables. Instead of fitting a linear model, let's fit a polynomial model of specified degree.**
- **scikit-learn provides various "transformers", which can transform the dataset for scaling, normalization, encoding categorical variables, filling missing values, and more. To perform polynomial regression, we will use the `PolynomialFeatures` class**.
- **scikit-learn also allows to create pipeline using `Pipeline` or `make_pipeline`, which are compositions of transformers followed by any model. Our polynomial regression model is a pipeline which first creates new features and then fits a linear regression model.**

**More information is available [here](https://scikit-learn.org/stable/data_transforms.html).**

**Create your model using the given `PolynomialRegression` function with `degree=2`. Then, plot the predictions of the model conditional to 'Weight' between 0 and 1250.**

In [7]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(SimpleImputer(strategy='mean'), PolynomialFeatures(degree), LinearRegression(**kwargs))

**5) We will now evaluate our first model with 10-fold cross-validation using the [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) function. In this process, the dataset is divided into 10 folds. For each fold, `cross_validate` will fit the model on the 9 other folds and test it on the remaining fold.**
- **Create a pipeline, composed of the `SimpleImputer` transformer from earlier and a `PolynomialRegression` model with degree 2.**
- **For evaluation, we will use the mean squared error (MSE). Note that scikit-learn provides [different score functions](https://scikit-learn.org/stable/modules/model_evaluation.html), which should be maximized. This is in contrast to loss functions, which should be minimized. Thus, we will use the score `neg_mean_squared_error` (the negation of the MSE), and we should not forget to take the negation when reporting it.**
- **Call `cross_validate` with 10 folds and specify the score function using the `scoring` argument. Since `X_train` and `y_train` have been modified previously with `SimpleImputer`, there will be data leakage between folds. Instead, use the given `X_train_raw` and `y_train_raw` as argument.**
- **Report the test MSE on each fold based on the dictionary of metrics returned by [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html). Then, report the mean test MSE across all fold.**

In [9]:
# We recreate the train and test sets because we modified the train set earlier.
X, y = df[['Weight']], df['Height']

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X, y, train_size=0.75, test_size=0.25, shuffle=True, random_state=1
)

**Let's now see how predictions vary with the model complexity. We go back to using the train and test sets defined earlier. For polynomial degrees between 1 and 5, we repeatedly fit the polynomial regression model (without cross-validation) and plot the predictions of 'Height' in function of the feature 'Weight'. What do you observe?**

In [11]:
models = []
degrees = [1, 2, 3, 5, 8, 15]
for degree in degrees:
    model = PolynomialRegression(degree, fit_intercept=True)
    model.fit(X_train, y_train)
    models.append(model)

fig, ax = plt.subplots()
ax.scatter(X_train, y_train, label='Train points', s=20)
X_plot = pd.DataFrame({'Weight': np.linspace(0, 2000, 500)})
for model in models:
    degree = model.get_params()['polynomialfeatures__degree']
    y_plot = model.predict(X_plot)
    ax.plot(X_plot, y_plot, label=f'Degree {degree}')
ax.set_xlim([X_train['Weight'].min() - 10, X_train['Weight'].max() + 10])
ax.set_ylim([y_train.min() - 1, y_train.max() + 1])
ax.set(xlabel='Weight', ylabel='Height')
fig.legend(loc='lower center', bbox_to_anchor=(0.5, 1), frameon=True, ncol=4)
fig.tight_layout()

**6) Collect and plot the evolution of the train and test MSE of the models from the previous question in function of the polynomial degree. Which model would you select?**

The polynomial of degree 2 achieves the lowest test MSE and should be selected.

**In the rest of this lab, we will predict the target 'Height' based on all available features. We split again the dataset into a training and test set following a 75/25 partition using the `train_test_split` function.**

In [13]:
X, y = df.drop(columns='Height'), df[['Height']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.75, test_size=0.25, shuffle=True, random_state=1
)

**7) We will now create a more complex pipeline to preprocess the data:**
- **For continuous variables, we replace the missing values by the sample mean.**
- **For missing categorical variables, replace missing value by the sample mode (the most frequent value).**
- **After replacing missing categorical variables, we replace them by dummy variables using the `OneHotEncoder`.**
- **To execute different transformers on different columns, we can use `ColumnTransformer`**

**For this, combine the transformers `SimpleImputer`, `OneHotEncoder` and `ColumnTransformer` into a single pipeline.**

In [14]:
# We can select continuous and categorical columns using this method
cont_columns = X_train.select_dtypes(include=['float64']).columns
cat_columns = X_train.select_dtypes(include=['category']).columns
print('Continuous columns:', cont_columns)
print('Categorical columns:', cat_columns)

In [15]:
# Transformers for imputation
cont_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_pipeline = make_pipeline(cat_imputer, OneHotEncoder(sparse_output=False, handle_unknown='ignore'))

# ColumnTransformer to apply transformations to the correct features
preprocessor = ColumnTransformer(transformers=[
    ('cont', cont_imputer, cont_columns),
    ('cat', cat_pipeline, cat_columns)
])

**8) Usually, models have more than one hyperparameter that can be tuned in order to find the model that best captures the relationship between our input and target variables. For instance, in the case of a simple linear regression using a polynomial transformation on the input variables, we can choose the hyperparameter space to be the polymial's degree, and whether or not to fit the intercept. Inspecting each combination of hyperparameters and selecting the combination that results in the best model is called grid search.**

**scikit-learn provides the class [`GridSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), which evaluates each combination of hyperparameters using cross-validation. After fitting, we can get relevant attributes:**
- **`cv_results_` returns a summary of the results and can be converted to a pandas dataframe`.**
- **`best_index_` returns the index of `cv_results_` for the best model.**
- **`best_params_` returns the parameters of the best model.**

**Perform a grid search on the hyperparameter space of a polynomial regression model (with all features). Search for degrees varying between 1 and 5, and whether or not the intercept should be fit. Report the best hyperparameters and the corresponding MSE.**

Perhaps surprisingly, the best score was obtained with a polynomial of degree 1. Although introducing more features offers greater flexibility to the model, it also raises the chance of the model fitting to noise within the features - an indication of overfitting.

**9) Finally, fit a model using the best hyperparameters on the full training dataset. You can use the function `set_params` of scikit-learn [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) classes. Collect the predictions on the test set in a variable `y_pred` and report the test MSE.**

We observe that the MSE decreases when the full training dataset is available.

**For visualization, we plot the predicted values of 'Height' as a function 'Weight'. Note that, due to the presence of other variables, the model is able to represent non-linear relationships between the two variables.**

In [20]:
# We sort according to the x-axis for visualization purposes
x_label = 'Weight'
X_test, y_test = X_test.reset_index(drop=True), y_test.reset_index(drop=True)
X_test = X_test.sort_values(x_label)
y_test = y_test.loc[X_test.index]
y_pred = y_pred[X_test.index]

fig, ax = plt.subplots()
ax.scatter(X_test[x_label], y_test, color='green', label='Test points', s=20)
ax.plot(X_test[x_label], y_pred, label='Best model found by cross-validation', color='r')
ax.set(xlabel=x_label, ylabel='Height')
ax.legend();

**Optional experiments: try to improve the performance of the model by manually transforming the features. For example add the squared 'Weight' as a feature.**